In [1]:
import torch
import math
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from torchtext.vocab import GloVe
from utils.preprocess import MINDIterator
from utils.utils import getVocab,getLoss,getLabel,constructBasicDict,run_eval,run_train
from models.FIM import FIMModel

## setting up the *FIM* model

### define paths and hyperparameters, load data

all these hyper parameters are fixed according to the paper [\[29\] Fine-grained Interest Matching for Neural News Recommendation](https://www.aclweb.org/anthology/2020.acl-main.77.pdf)

- *mode*: data to read (*small*/*large*)

- *batch_size*: size of each minibatch

- *title_size*: max word capacity of title

- *his_size*: max record capacity of click history

- *npratio*: number of negtive sampling

- *dilation_level*: levels of diferrent dilation rate

- *kernel_size*: size of 1dCNN kernel

- *filter_num*: number of kernels in 1D CNN, which is also embedding dimension of news/user

- *embedding_dim*: word embedding dimension

### Note 

Although I list *dilation_level* and *kernel_size*, some properties related to both variables are fixed in the FIM model because I don't have time to well-design my model to make it adapt to these variables dynamically. 

In [2]:
hparams = {
    'mode':'small',
    'batch_size':5,
    'title_size':18,
    'his_size':50,
    'kernel_size':3,
    'npratio':4,     
    'dilation_level':3,
    'filter_num':150,
    'embedding_dim':300,
    'metrics':'group_auc,ndcg@4,mean_mrr',
    'gpu':'cuda:0'
}

# customize your path here
news_file = r'D:\Data\NR_data\dev\news.tsv'
behavior_file_train = r'D:\Data\NR_data\dev\behaviors_train.tsv'
behavior_file_test = r'D:\Data\NR_data\dev\behaviors_test.tsv'
save_path = 'models/model_param/FIM.model'

# if user2id,word2id,news2id hasn't been constructed
if not os.path.exists('data/vocab_'+hparams['mode']+'.pkl'):
    constructBasicDict(news_file,behavior_file_train,hparams['mode'])

device = torch.device(hparams['gpu']) if torch.cuda.is_available() else torch.device("cpu")

iterator_train = MINDIterator(hparams=hparams,news_file=news_file,behaviors_file=behavior_file_train)

vocab = iterator_train.vocab
embedding = GloVe(dim=300,cache='.vector_cache')
vocab.load_vectors(embedding)

In [3]:
# you can load my model or train yours
if os.path.exists(save_path):
    fimModel = FIMModel(vocab=vocab,hparams=hparams)
    fimModel.load_state_dict(torch.load(save_path))
    fimModel.to(device).eval()

else:
    fimModel = FIMModel(vocab=vocab,hparams=hparams).to(device)
    fimModel.train()

### train the model

In [4]:
if fimModel.training:
    loss_func = getLoss(fimModel)
    optimizer = optim.Adam(fimModel.parameters(),lr=0.001)
    fimModel = run_train(fimModel,iterator_train,optimizer,loss_func)

epoch 0 , step 100 , total_loss: 1.6062, batch_loss: 1.5703: : 143it [00:09, 14.69it/s]
epoch 1 , step 100 , total_loss: 1.4956, batch_loss: 1.4249: : 143it [00:07, 19.12it/s]
epoch 2 , step 100 , total_loss: 1.4507, batch_loss: 1.3582: : 143it [00:07, 19.00it/s]
epoch 3 , step 100 , total_loss: 1.3372, batch_loss: 1.2950: : 143it [00:07, 18.75it/s]
epoch 4 , step 100 , total_loss: 1.1739, batch_loss: 1.3782: : 143it [00:07, 18.63it/s]
epoch 5 , step 100 , total_loss: 1.1386, batch_loss: 1.8118: : 143it [00:07, 18.54it/s]
epoch 6 , step 100 , total_loss: 1.0365, batch_loss: 1.5139: : 143it [00:07, 18.56it/s]
epoch 7 , step 100 , total_loss: 0.8622, batch_loss: 0.8031: : 143it [00:07, 18.40it/s]
epoch 8 , step 100 , total_loss: 0.8085, batch_loss: 0.9867: : 143it [00:07, 18.37it/s]
epoch 9 , step 100 , total_loss: 0.6580, batch_loss: 0.5094: : 143it [00:07, 18.41it/s]


### test & evaluate

In [4]:
iterator_test = MINDIterator(hparams,news_file,behavior_file_test)
iterator_test.npratio = -1
test = iterator_test.load_data_from_file()

fimModel.eval()
fimModel.npratio = -1
run_eval(fimModel,test)

{'group_auc': 0.6386, 'ndcg@4': 0.2424, 'mean_mrr': 0.2528}

### save the model

In [9]:
fimModel.npratio = 4
torch.save(fimModel.state_dict(), save_path)